In [13]:
import pandas as pd
df = pd.read_parquet("/Users/arjungupta/Downloads/train-00000-of-00001.parquet")
df = df.fillna("")

In [10]:
import redis
from redis.commands.search.field import TextField
from redis.commands.search.index_definition import IndexDefinition, IndexType

In [8]:
r = redis.Redis(host="localhost", port=6379, decode_responses=True)

In [11]:
try:
    r.ft("games").create_index(
        fields=[
            TextField("search_text"),
            TextField("Name"),
            TextField("Genre"),
            TextField("Publisher")
        ],
        definition=IndexDefinition(prefix=["game:"], index_type=IndexType.HASH)
    )
except Exception as e:
    print("Index creation failed or already exists:", e)

In [14]:
for idx, row in df.iterrows():
    game_id = f"game:{idx}"
    
    search_text = f"{row['Name']} {row['Genre']} {row['Publisher']}"
    
    r.hset(game_id, mapping={
        "search_text": search_text,
        "Name": row["Name"],
        "Genre": row["Genre"],
        "Publisher": row["Publisher"]
    })